
# 🌍 Manual TOA Reflectance from Landsat 8

## 🎯 Learning Objectives:
- Load Landsat 8 Level-1 data from Earth Engine
- Understand the process of converting DN values to radiance
- Compute Top-of-Atmosphere (TOA) reflectance from radiance
- Visualize results on an interactive map using `geemap`

---

## 📝 Your Tasks:
1. Load a Landsat 8 Level-1 image over Kraków in July 2021.
2. From metadata, extract the scaling coefficients for each band (RADIANCE_MULT, RADIANCE_ADD).
3. Convert DN to Radiance using the linear formula.
4. Use the ESUN values and solar elevation to convert Radiance to TOA Reflectance.
5. Visualize both the original DN RGB and your TOA Reflectance RGB image.
6. (Optional) Try using a different location or time range.



## 🗺️ Step 1: Define Area of Interest (AOI)

Use Kraków (Poland) as your starting point. You can define this using a point geometry.


In [1]:
import ee
import geemap
import math

ee.Authenticate()
ee.Initialize(project='forevery32')

aoi = ee.Geometry.Rectangle([19.959294, 50.086612, 19.992467, 50.103602])


## 📦 Step 2: Load Landsat 8 Level-1 Image (Collection 2)

Load the `LANDSAT/LC08/C02/T1_L1` collection, filter for July 2021, and select the image with the least cloud cover.


In [2]:
image = ee.ImageCollection("LANDSAT/LC08/C02/T1").filterBounds(aoi).filterDate('2021-08-01', '2021-08-31').sort('CLOUDY_PIXEL_PERCENTAGE').first()
image


## 🔢 Step 3: Convert DN to Radiance

Use the formula:

```
Radiance = M_L * DN + A_L
```

Where:
- `M_L` is `RADIANCE_MULT_BAND_X` from metadata
- `A_L` is `RADIANCE_ADD_BAND_X`
- DN is the digital number of each band

💡 Tip: You can use `image.get('RADIANCE_MULT_BAND_4')` to retrieve the multiplier for band 4.


In [3]:
import ee

def radiance_calculator(image, band):
    
    M_L = ee.Number(image.get(f'RADIANCE_MULT_BAND_{band}'))
    A_L = ee.Number(image.get(f'RADIANCE_ADD_BAND_{band}'))
    DN = image.select(f'B{band}')
    

    print("M_L (Multiplier):", M_L.getInfo())
    print("A_L (Addend):", A_L.getInfo())

    radiance = DN.multiply(M_L).add(A_L).rename(f'B{band}_rad')
    
    return radiance



## 📊 Step 4: Convert Radiance to TOA Reflectance

Use the following equation from the USGS Landsat Handbook:

```
TOA Reflectance = (π * L_λ * d²) / (ESUN * cos(θ_s))
```

Where:
- `L_λ` = radiance
- `d` = Earth–Sun distance in AU (assume 1.0 for simplification)
- `ESUN` = solar spectral irradiance (use constants from handbook)
- `θ_s` = solar zenith angle = 90° - sun elevation (from image metadata)

Use these ESUN values:
- B2: 2067  
- B3: 1893  
- B4: 1603  
- B5: 972


In [4]:
import ee
import math

def toa_reflectance_calculator(image, band):
    """
    Convert Radiance to TOA Reflectance for a specific band.

    Args:
        image (ee.Image): The input image.
        band (int): The band number (e.g., 4 for Band 4).

    Returns:
        ee.Image: TOA Reflectance image for the specified band.
    """
    # Step 1: Get Radiance
    radiance = radiance_calculator(image, band)  # reuse your radiance function

    # Step 2: Define ESUN values
    esun_values = {
        'B2': 2067,
        'B3': 1893,
        'B4': 1603,
        'B5': 972
    }
    ESUN = ee.Number(esun_values[band])

    # Step 3: Earth-Sun distance (assumed 1.0 for simplicity)
    d = ee.Number(1.0)

    # Step 4: Get Solar Zenith Angle
    sun_elevation = ee.Number(image.get('SUN_ELEVATION'))
    theta_s = ee.Number(90).subtract(sun_elevation)
    cos_theta_s = theta_s.multiply(math.pi/180).cos()  # Convert degrees to radians

    # Step 5: Apply TOA Reflectance Formula
    toa_reflectance = (radiance.multiply(math.pi)
                              .multiply(d.pow(2))
                              .divide(ESUN.multiply(cos_theta_s))
                              .rename(f'B{band}_toa'))

    return toa_reflectance

    toa_band1 = toa_reflectance_calculator(image, 'B2')
    toa_band2 = toa_reflectance_calculator(image, 'B3')
    toa_band3 = toa_reflectance_calculator(image, 'B4')
    #toa_band4 = toa_reflectance_calculator(image, 'B5')





## 🖼️ Step 5: Visualize the Results

Use `geemap` to create an interactive map. Display:
- Original RGB from DN bands
- Your manually calculated TOA Reflectance RGB

💡 Hint: Use `.clamp(0, 0.4)` on reflectance bands for better visualization.


In [ ]:
rgb_bands = image.select([ 'B4', 'B3', 'B2'])

toa_band1 = image.select('B4').multiply(0.0001)  # Example TOA reflectance calculation
toa_band2 = image.select('B3').multiply(0.0001)  # Example TOA reflectance calculation
toa_band3 = image.select('B2').multiply(0.0001)  # Example TOA reflectance calculation
#toa_band4 = image.select('B5').multiply(0.0001)  # Example TOA reflectance calculation

toa_reflectance = ee.Image.cat([toa_band1, toa_band2, toa_band3])
clamped_reflectance = toa_reflectance.clamp(0, 0.4)

#rgb_toa = ee.Image.cat([toa_band1.rename('B1_toa'), toa_band2.rename('B2_toa'), toa_band3.rename('B3_toa'), toa_band4.rename('B4_toa')])
#rgb_toa = rgb_dn.clamp(0, 0.4)

Map = geemap.Map(center=[50.], zoom=10)
Map.addLayer(rgb_bands, {'min': 0, 'max': 3000, 'bands': ['B4', 'B3', 'B2']}, 'Original RGB')
Map.addLayer(clamped_reflectance, {'min': 0, 'max': 0.4, 'bands': ['B4', 'B3', 'B2']}, 'TOA Reflectance RGB')

#Map.addLayer(rgb_dn, {'min': 0, 'max': 10000}, 'RGB Image')
Map.addLayer(clamped_reflectance, {'min': 0, 'max': 0.4}, 'TOA Reflectance RGB')
Map

EEException: Caller does not have required permission to use project 517222506229. Grant the caller the roles/serviceusage.serviceUsageConsumer role, or a custom role with the serviceusage.services.use permission, by visiting https://console.developers.google.com/iam-admin/iam/project?project=517222506229 and then retry. Propagation of the new permission may take a few minutes.


## 🧪 Optional Extensions

- Try the same workflow on a different city or biome (e.g., desert, rainforest).
- Add NIR band to your analysis (B5).
- Compute NDVI from TOA or compare with Surface Reflectance data.
- Export the resulting reflectance image to Google Drive.
